In [5]:
import pandas as pd
import numpy as np
import random
import time
import os
import re
from tqdm import tqdm 
from datetime import datetime

import spacy
from spacy.matcher import Matcher

In [6]:
def get_languages(matcher, nlp, text):
    languages = [[{"LOWER": {"IN": ['apl', 'assembly', 'bash', 'shell', 'c++', 'c#', 'clojure',
                          'cobol', 'crystal', 'dart', 'delphi', 'elixir', 'haskell',
                          'java', 'javascript', 'julia', 'matlab', 'php','powershell', 'python',
                          'r', 'ruby', 'rust', 'sas', 'solidity', 'sql', 'vba','typescript', 'html', 'css',
                          'swift']}}],
            [{"LOWER":"c"}, {"ORTH":{"IN": ['#', "++"]}}]]
    
    for language in languages:
        matcher.add("LanguagePattern", [language])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    languages = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        languages.append(matched_text)
    languages = list(set(languages))
    languages = ", ".join(languages)
    return languages if languages != '' else None

def get_databases(matcher, nlp, text):
    
    databases = [[{"LOWER": {"IN": ['cassandra', 'couchbase', 'couchdb', 'dynamodb', 'elasticsearch', 
                                 'mariadb', 'mongodb', 'mysql', 'neo4j', 'oracle', 'postgresql',
                                 'redis', 'sqlite', 'db2']}}],
               [{"LOWER": "cloud"}, {"LOWER": 'firestone'}],
               [{"LOWER": "microsoft"}, {"LOWER": "sql"}, {"LOWER": "server"}],
               [{"LOWER": "firebase"}, {"LOWER": "realtime"}, {"LOWER": "database"}]]
    
    for database in databases:
        matcher.add("DatabasePattern", [database])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    database = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        database.append(matched_text)
    database = list(set(database))
    database = ", ".join(database)
    return database if database != '' else None

def get_cloud_platforms(matcher, nlp, text):
    cloud_platforms = [[{"LOWER": {"IN": ['aws', 'amazon', 'colocation', 'digitanocean', 'firebase',
                                        'heroku', 'linode', 'openstack', 'azure', 'gcp', 'ovh', 'vmware']}}],
                     [{"LOWER":'amazon'}, {"LOWER": "web"}],
                     [{"LOWER": 'google'}, {"LOWER": 'cloud'}],
                     [{"LOWER": 'microsoft'}, {"LOWER": 'azure'}],
                     [{'LOWER': "ibm"}, {"LOWER": 'cloud'}],
                     [{"LOWER": 'managed'}, {"LOWER": "hosting"}],
                     [{"LOWER": "oracle"}, {"LOWER": "cloud"}]]
    
    for cloud_platform in cloud_platforms:
        matcher.add("CloudPattern", [cloud_platform])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    cloud_platform = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        cloud_platform.append(matched_text)
    
    final_cloud = []
    for cloud in cloud_platform:
        if (cloud == 'aws') | (cloud=='amazon') | (cloud == "amazon web"):
            final_cloud.append("amazon web services")
        elif (cloud == 'azure') | (cloud == 'microsoft azure'):
            final_cloud.append('microsoft azure')
        elif (cloud == 'gcp') | (cloud == 'google cloud'):
            final_cloud.append("google cloud product")
        else:
            final_cloud.append(cloud)
            
    final_cloud = list(set(final_cloud))
    final_cloud = ", ".join(final_cloud)
    return final_cloud if final_cloud != '' else None
 
def get_web_frameworks(matcher, nlp, text):
    web_frameworks = [[{"LOWER": {"IN": ['angular', 'django', 'drupal', 'fastapi', 'fastify',
                                   'flask', 'jquery', 'gatsby', 'laravel', 'phoenix',
                                   'svelte', 'symfony', 'nodejs', 'nextjs', 'aspnet', 
                                   'vuejs', 'reactjs']}}]]
    for web_framework in web_frameworks:
        matcher.add("WebFrameworkPattern", [web_framework])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    web_framework = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        web_framework.append(matched_text)
    web_framework = list(set(web_framework))
    web_framework = ", ".join(web_framework)
    return web_framework if web_framework != '' else None

def get_libraries(matcher, nlp, text):
    libraries = [[{"LOWER": {"IN": ['kafka', 'spark', 'capacitor', 'electron', 'flutter',
                               'gtk', 'hadoop', 'ionic', 'keras', 'numpy', 'pandas',
                               'qt', 'scikit-learn', 'tensorflow', 'tidyverse',
                               'pytorch', 'pyspark', 'xamarin']}}],
            [{"LOWER": 'react'}, {"LOWER":"native"}],
            [{"LOWER": "hugging"}, {"LOWER": "face"}],
            [{"LOWER": "uno"}, {"LOWER": "platform"}]]
    
    for library in libraries:
        matcher.add("LibraryPattern", [library])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    library = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        library.append(matched_text)
    library = list(set(library))
    library = ", ".join(library)
    return library if library != '' else None

def get_developers(matcher, nlp, text):
    developers = [[{"LOWER" : {"IN": ['ansible', 'chef', 'docker', 'homebrew', 'kubernetes',
                                 'npm', 'pulumi', 'pupper', 'terraform', 'yarn']}}],
             [{"LOWER": "unity"}, {"LOWER": "3d"}],
             [{"LOWER": "unreal"}, {"LOWER": "engine"}]]
    
    for developer in developers:
        matcher.add("DeveloperPattern", [developer])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    developer = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        developer.append(matched_text)
    developer = list(set(developer))
    developer = ", ".join(developer)
    return developer if developer != '' else None

def get_data_visualizations(matcher, nlp, text):
    data_viz = [[{"LOWER": {"IN": ['looker', 'tableau', 'qlik', 'powerbi']}}],
               [{"LOWER": "power"}, {"LOWER": 'bi'}]]
    
    for data_vi in data_viz:
        matcher.add("DataVisualizationPattern", [data_vi])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    data_vi = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        data_vi.append(matched_text)
        
    final_data_viz = []
    for viz in data_vi:
        if (viz == "powerbi") | (viz == "power bi"):
            final_data_viz.append("power bi")
        else:
            final_data_viz.append(viz)
            
    final_data_viz = list(set(final_data_viz))
    final_data_viz = ", ".join(final_data_viz)
    return final_data_viz if final_data_viz != '' else None

def get_version_controls(matcher, nlp, text):
    version_controls = [[{"LOWER": {"IN": ['bitbucket', 'github', 'gitlab']}}]]
    
    for version_control in version_controls:
        matcher.add("VersionControlPattern", [version_control])
        
    doc = nlp(text)
    matches = matcher(doc)
    
    version_control = []
    for match_id, start, end in matches:
        matched_text = doc[start:end].text.lower()
        version_control.append(matched_text)
    version_control = list(set(version_control))
    version_control = ", ".join(version_control)
    return version_control if version_control != '' else None

def matcher(df):
    nlp = spacy.load('en_core_web_sm')

    language_matcher = Matcher(nlp.vocab)
    database_matcher = Matcher(nlp.vocab)
    cloud_matcher = Matcher(nlp.vocab)
    web_framework_matcher = Matcher(nlp.vocab)
    library_matcher = Matcher(nlp.vocab)
    developer_matcher = Matcher(nlp.vocab)
    data_viz_matcher = Matcher(nlp.vocab)
    version_control_matcher = Matcher(nlp.vocab)

    for i, row in tqdm(df.iterrows()):
        text = row['summary']
        
        df.loc[i, ['languages', 'databases', 'cloud_platforms', 'web_frameworks',
                    'libraries', 'developers', 'data_visualization', 'version_controls']] = [get_languages(language_matcher, text),
                                                                                                get_databases(database_matcher, text),
                                                                                                get_cloud_platforms(cloud_matcher, text),
                                                                                                get_web_frameworks(web_framework_matcher, text),
                                                                                                get_libraries(library_matcher, text),
                                                                                                get_developers(developer_matcher, text),
                                                                                                get_data_visualizations(data_viz_matcher, text),
                                                                                                get_version_controls(version_control_matcher, text)
                                                                                                ]
    return df

In [7]:
data = pd.read_csv(r"C:\Users\Emerson\jobstreet_data_engineer_dashboard\data\jobstreet-2023-09-22.csv")

In [9]:
nlp = spacy.load("en_core_web_sm")
data = matcher(data)

725it [07:57,  1.52it/s]


In [10]:
data

,job_id,location,title,company,salary,summary,job_url,date_posted,Career Level,Qualification,...,Registration No.,POEA No.,languages,databases,cloud_platforms,web_frameworks,libraries,developers,data_visualization,version_controls
0,13237392,Quezon City,Data Engineer,Manulife Business Processing Services (MBPS),NaN,Are you looking for a supportive and collabora...,https://www.jobstreet.com.ph/en/job/data-engin...,2023-09-22,1-4 Years Experienced Employee,Bachelor's/College Degree,...,NaN,NaN,"sql, python, shell",mongodb,microsoft azure,NaN,"spark, hadoop, kafka",NaN,"power bi, tableau",NaN
1,13228347,Taguig City,Data Engineer - Cavite,Schneider Electric,"PHP 70,000 - PHP 98,000",Great People make Schneider Electric a great c...,https://www.jobstreet.com.ph/en/job/data-engin...,2023-09-22,1-4 Years Experienced Employee,Not Specified,...,NaN,NaN,sql,"oracle, mysql, postgresql",amazon web services,NaN,spark,NaN,"power bi, tableau",NaN
2,13235504,Multiple work locations,Snowflake Data Engineer,Accenture,NaN,Ready to become part of the dynamic Accenture ...,https://www.jobstreet.com.ph/en/job/snowflake-...,2023-09-22,1-4 Years Experienced Employee,"Bachelor's/College Degree, Post Graduate Diplo...",...,NaN,NaN,"sql, python, java",None,"microsoft azure, amazon web services",NaN,"spark, hadoop, kafka",NaN,"power bi, looker, tableau",bitbucket
3,13237413,National Capital Reg,Data Engineer | Day Shift | Permanent WFH | Da...,Emapta,NaN,Join a Fashion Retail Revolution – Data Engine...,https://www.jobstreet.com.ph/en/job/data-engin...,2023-09-22,1-4 Years Experienced Employee,Bachelor's/College Degree,...,NaN,NaN,"sql, python, java",None,microsoft azure,NaN,None,NaN,power bi,None
4,13238314,National Capital Reg,Data Engineer (Azure Data Platforms),Emerson Electric (Asia) Limited - ROHQ,NaN,Emerson’s 132 years of history have been fille...,https://www.jobstreet.com.ph/en/job/data-engin...,2023-09-22,1-4 Years Experienced Employee,Bachelor's/College Degree,...,NaN,NaN,"sql, powershell",None,microsoft azure,NaN,None,NaN,power bi,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,10368334,National Capital Reg,Service Desk Analyst,WHITE & CASE-GLOBAL OPERATIONS CENTER (MANILA)...,NaN,Firm Summary\nWhite & Case is an elite global ...,https://www.jobstreet.com.ph/en/job/service-de...,2023-09-22,Not Specified,Not Specified,...,NaN,NaN,None,None,None,None,None,None,None,None
721,10368922,Makati City,Multi-skilled Technician,"Jones Lang LaSalle Phils., Inc.",NaN,"JLL supports the Whole You, personally and pro...",https://www.jobstreet.com.ph/en/job/multi-skil...,2023-09-22,Not Specified,Not Specified,...,NaN,NaN,None,None,None,None,spark,None,None,None
722,10368923,National Capital Reg,Multi-skilled Technician,"Jones Lang LaSalle Phils., Inc.",NaN,"JLL supports the Whole You, personally and pro...",https://www.jobstreet.com.ph/en/job/multi-skil...,2023-09-22,Not Specified,Not Specified,...,NaN,NaN,None,None,None,None,spark,None,None,None
723,10368209,National Capital Reg,Accountant - Payroll,First Datacorp,NaN,"Our purpose\nHere at Datacom, we connect peopl...",https://www.jobstreet.com.ph/en/job/accountant...,2023-09-22,Not Specified,Not Specified,...,NaN,NaN,None,oracle,None,None,None,None,None,None
